In [ ]:
# Imports
import os
import torch.cuda, torch.utils.data, torch.nn, torch.optim, torch
import torchvision.transforms, torchvision.datasets.folder, torchvision.utils
import cv2
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import time

In [ ]:
# Addresses

class Address:
    def __init__(self):
        '''
        Stores all the addresses used in project
        '''
        # Inputs
        self.data = "../input/mammography"

        # Models
        self.result = "results/"

        # Temp
        self.temp = "temp/"

    def create_dir(self, dir_list = None):
        '''
        Function to create directories in dir_list. If dir_list is None then create all directories of address.
        '''
        if dir_list == None:
            dir_list = [self.temp]
        for address in dir_list:
            if not os.path.exists(address):
                os.mkdir(address)

    def _delete_folder_content(self, folder_addr):
        '''
        Deletes all the content of folder_addr
        '''
        if os.path.exists(folder_addr):
            for file in os.listdir(folder_addr):
                address = os.path.join(folder_addr, file)
                if os.path.isdir(address):
                    self._delete_folder_content(address)
                    os.removedirs(address)
                else:
                    os.remove(address)

    def clean(self, file_list = None):
        '''
        Deletes all the content in file_list
        '''
        if file_list == None:
            file_list = [self.temp]
        for address in file_list:
            self._delete_folder_content(address)

addr = Address()
addr.clean()
addr.create_dir()

In [ ]:
class HyperParameters:
    def __init__(self):
        '''
        Stores all Hyperparameters used for training of model
        '''
        # Training
        self.batch_size = 32
        self.num_epoch = 50
        self.grad_clip = 1.0

        # Data
        self.num_train = 40000
        self.num_val = 10000
        self.train_step = self.num_epoch*(self.num_train//self.batch_size)
        
        # Learning Rate
        self.lr = 4e-4
        self.warmup_step = self.train_step//40
        self.decay_step = self.train_step//2
        self.decay_rate = 0.5

    def lr_schedule(self, step):
        '''
        Getting learning rate as function of train steps completed (Exponential decay with linear warmup)
        '''
        if step <= self.warmup_step:
            return step/self.warmup_step
        else:
            return self.decay_rate**((step-self.warmup_step)/self.decay_step)

    def create_report(self, addr):
        with open(os.path.join(addr, 'param.txt'), 'w') as file:
            file.writelines([
                f'Training:',
                f'\n\tBatch Size:       {self.batch_size}',
                f'\n\tNum Epoch:        {self.num_epoch}',
                f'\n\tGrad Clip:        {self.grad_clip}',
                f'\n\nData:',  
                f'\n\tNum Train:        {self.num_train}',
                f'\n\tNum Val:          {self.num_val}',
                f'\n\tTrain Step:       {self.train_step}',
                f'\n\nLearning Rate:',  
                f'\n\tlr:               {self.lr}',
                f'\n\tWarmup Step:      {self.warmup_step}',
                f'\n\tDecay Step:       {self.decay_step}',
                f'\n\tDecay Rate:       {self.decay_rate}',
            ])

param = HyperParameters()

In [ ]:
# Random Seed and CUDA

random_seed = 68
device = "cpu"
torch.manual_seed(random_seed)
np.random.seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
    device = "cuda"
print(f"Working with device {device}")